In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import os
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import numpy as np
import cv2
import mediapipe as mp

In [2]:
mp_hands = mp.solutions.hands.Hands(
    max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5
)

In [7]:
def process_image_with_mediapipe(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = mp_hands.process(image_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            min_x, min_y, max_x, max_y = image.shape[1], image.shape[0], 0, 0
            for landmark in hand_landmarks.landmark:
                x, y = int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0])
                min_x = min(min_x, x)
                max_x = max(max_x, x)
                min_y = min(min_y, y)
                max_y = max(max_y, y)
            cropped_image = image[min_y:max_y, min_x:max_x]
            return cropped_image
    return None

In [8]:
data_dir = 'C:/Sign Language Detection/Module 1 (sign to text)/dataset/'
output_dir = 'C:/Sign Language Detection/Module 1 (sign to text)/processed_dataset/'

os.makedirs(output_dir, exist_ok=True)

In [9]:
for phase in ['Train_Alphabet', 'Test_Alphabet']:
    print(f"Processing {phase}...")
    output_phase_dir = os.path.join(output_dir, phase)
    os.makedirs(output_phase_dir, exist_ok=True)

    for root, _, filenames in os.walk(os.path.join(data_dir, phase)):
        for filename in filenames:
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(root, filename)
                processed_image = process_image_with_mediapipe(image_path)
                if processed_image is not None:
                    output_path = os.path.join(output_phase_dir, os.path.relpath(image_path, data_dir))
                    os.makedirs(os.path.dirname(output_path), exist_ok=True)
                    cv2.imwrite(output_path, processed_image)

print("Hand images dataset created successfully.")

Processing Train_Alphabet...
Processing Test_Alphabet...
Hand images dataset created successfully.
